<img style="float: left;" src="./images/logo.png" width="400"/>

this code is from the official GFED website

In [ ]:
months       = '01','02','03','04','05','06','07','08','09','10','11','12'
sources      = 'SAVA','BORF','TEMF','DEFO','PEAT','AGRI'

in this example we will calculate annual CO emissions for the 14 GFED basisregions over 1997-2014. 

Please adjust the code to calculate emissions for your own specie, region, and time period of interest. 

Please first download the GFED4.1s files (available on the dropbox folder for simplicity) and the GFED4_Emission_Factors_Summary.csv to your computer and adjust the directory where you placed them below

In GFED4.1s biomass burning dry matter (DM) emissions are provided in kg DM/m2/month in the yearly file `GFED4.1s_xxxx.hdf5`. This corresponds to the mass of burnt fuel

## Read in emission factors

In [ ]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

In [ ]:
directory    = './GFED4.1s/data/'

In [ ]:
species = [] # names of the different gas and aerosol species
EFs     = np.zeros((41, 6)) # 41 species, 6 sources

data_EF = pd.read_csv(directory+'/GFED4_Emission_Factors_Summary.csv', header=10)
EF_CO = np.array(data_EF.iloc[3])[1:]


# we are interested in CO for this example (4th row):

start_year = 1997
end_year   = 2014

## Make table with summed DM emissions for each region, year, and source

In [ ]:

CO_table = np.zeros((15, end_year - start_year + 1)) # region, year

for year in range(start_year, end_year+1):
    string = directory+'/GFED4.1s_'+str(year)+'.hdf5'
    f = h5py.File(string, 'r')
    
    
    if year == start_year: # these are time invariable    
        basis_regions = f['/ancill/basis_regions'][:]
        grid_area     = f['/ancill/grid_cell_area'][:]
    
    
    CO_emissions = np.zeros((720, 1440))
    for month in range(12):
        # read in DM emissions
        string = '/emissions/'+months[month]+'/DM'
        DM_emissions = f[string][:]
        for source in range(6):
            # read in the fractional contribution of each source
            string = '/emissions/'+months[month]+'/partitioning/DM_'+sources[source]
            contribution = f[string][:]
            # calculate CO emissions as the product of DM emissions (kg DM per 
            # m2 per month), the fraction the specific source contributes to 
            # this (unitless), and the emission factor (g CO per kg DM burned)
            CO_emissions += DM_emissions * contribution * float(EF_CO[source])
    
    
    # fill table with total values for the globe (row 15) or basisregion (1-14)
    for region in range(15):
        if region == 14:
            mask = np.ones((720, 1440))
        else:
            mask = basis_regions == (region + 1)            
    
        CO_table[region, year-start_year] = np.sum(grid_area * GFED_example/load_GFED_data_example.ipynbmask * CO_emissions)
        
    print(year)
        
# convert to Tg CO 
CO_table = CO_table / 1E12
print(CO_table)

# please compare this to http://www.falw.vu/~gwerf/GFED/GFED4/tables/GFED4.1s_CO.txt

## To go further

Using the result from the last exercise of the introductionary python lecture (see [here](https://github.com/3dfirelab/python-intro-course/blob/master/06-pandasExcercise.ipynb) can you calculate total carbone emission for you country? 